## 로지스틱 회귀

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

fish = pd.read_csv('https://bit.ly/fish_csv_data')
fish.head(10)


,Species,Weight,Length,Diagonal,Height,Width
0,Bream,242.0,25.4,30.0,11.5200,4.0200
1,Bream,290.0,26.3,31.2,12.4800,4.3056
2,Bream,340.0,26.5,31.1,12.3778,4.6961
3,Bream,363.0,29.0,33.5,12.7300,4.4555
4,Bream,430.0,29.0,34.0,12.4440,5.1340
5,Bream,450.0,29.7,34.7,13.6024,4.9274
6,Bream,500.0,29.7,34.5,14.1795,5.2785
7,Bream,390.0,30.0,35.0,12.6700,4.6900
8,Bream,450.0,30.0,35.1,14.0049,4.8438
9,Bream,500.0,30.7,36.2,14.2266,4.9594


In [2]:
fish_input = fish[['Weight','Length','Diagonal','Height','Width']].to_numpy()
fish_target = fish['Species'].to_numpy()

In [5]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(fish_input, fish_target, random_state=42)

In [6]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
ss.fit(train_input)

train_scaled = ss.transform(train_input)
test_scaled = ss.transform(test_input)

In [10]:
from sklearn.neighbors import KNeighborsClassifier
kn = KNeighborsClassifier(n_neighbors=3)
kn.fit(train_scaled, train_target)

print(kn.classes_)
print(kn.predict(test_scaled[:5]))

proba = kn.predict_proba(test_scaled[:5])  ## 확률을 출력하는 함수 kn.predict_proba
print(np.round(proba, decimals=4))
print(kn.score(train_scaled, train_target))

['Bream' 'Parkki' 'Perch' 'Pike' 'Roach' 'Smelt' 'Whitefish']
['Perch' 'Smelt' 'Pike' 'Perch' 'Perch']
[[0.     0.     1.     0.     0.     0.     0.    ]
 [0.     0.     0.     0.     0.     1.     0.    ]
 [0.     0.     0.     1.     0.     0.     0.    ]
 [0.     0.     0.6667 0.     0.3333 0.     0.    ]
 [0.     0.     0.6667 0.     0.3333 0.     0.    ]]
0.8907563025210085


  - 사이킷런에서 변수뒤에 '_' 이 붙는 다면 사용자가 지정한 변수명이 아닌 모델이 스스로 학습하면서 얻게되는 정보를 의미한다.

### 로지스틱 회귀  = 분류알고리즘

z = a* 무게 + b* 길이 + c* 대각선 + d*높이 + e* 두께 + f

π = 1 / (1+e^-z)

### 로지스틱 회귀(이진분류)


In [11]:
bream_smelt_indexes = (train_target == 'Bream') | (train_target == 'Smelt')   ## 블리언 인덱싱
train_bream_smelt = train_scaled[bream_smelt_indexes]
target_bream_smelt = train_target[bream_smelt_indexes]

from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(train_bream_smelt, target_bream_smelt)

print(lr.predict(train_bream_smelt[:5]))

['Bream' 'Smelt' 'Bream' 'Bream' 'Bream']


In [12]:
print(lr.predict_proba(train_bream_smelt[:5]))

[[0.99759855 0.00240145]
 [0.02735183 0.97264817]
 [0.99486072 0.00513928]
 [0.98584202 0.01415798]
 [0.99767269 0.00232731]]


In [13]:
print(target_bream_smelt)

['Bream' 'Smelt' 'Bream' 'Bream' 'Bream' 'Smelt' 'Bream' 'Bream' 'Bream'
 'Bream' 'Bream' 'Bream' 'Bream' 'Smelt' 'Bream' 'Smelt' 'Smelt' 'Bream'
 'Bream' 'Bream' 'Bream' 'Bream' 'Bream' 'Bream' 'Bream' 'Smelt' 'Bream'
 'Smelt' 'Smelt' 'Bream' 'Smelt' 'Bream' 'Bream']


### 로지스틱 회귀 계수 확인

In [14]:
print(lr.coef_, lr.intercept_)

[[-0.4037798  -0.57620209 -0.66280298 -1.01290277 -0.73168947]] [-2.16155132]


  -  z = -0.404 * 무게 -0.576 * 길이 - 0.663 * 대각선 - 0.013 * 높이 - 0.732 * 두께 - 2.161

In [15]:
decisions = lr.decision_function(train_bream_smelt[:5])
print(decisions)

[-6.02927744  3.57123907 -5.26568906 -4.24321775 -6.0607117 ]


In [16]:
from scipy.special import expit  ##  scipy 라이브러리에 시그모이드 함수를 활용해서 구할 수 있다.

print(expit(decisions))

[0.00240145 0.97264817 0.00513928 0.01415798 0.00232731]


## 로지스틱 회귀(다중분류)

In [34]:
lr = LogisticRegression(C=20, max_iter=1000)
lr.fit(train_scaled, train_target)

print(lr.score(train_scaled, train_target))
print(lr.score(test_scaled, test_target))

0.9327731092436975
0.925


  - C 값은 규제를 의미하며 C 값이 높아지면 규제가 약해지고(과적합이 될 가능성이 존재), C값이 작아지면 규제가 강해진다(과소적합이 발생할수있음).
  - 사이킷런에서의 로지스틱함수는 L2 규제를 기본으로 사용

  - 𝜆(람다(lambda))는 머신 러닝과 통계학에서 정규화 파라미터(regularization parameter) 또는 규제 항(regularization term)이라고 부릅니다. 이는 모델의 복잡도를 조절하고 과적합을 방지하기 위해 추가되는 항입니다.
  - C 값은 𝜆의 역수 입니다.

  - LinearRegression 에서의 alpha 변수값(규제의 강도를 의미) 과 동일하지만 서로 방향이 반대이다

  - C값의 default 값은 1이다. (LinearRegression의 alpha와 동일)

In [35]:
proba = lr.predict_proba(test_scaled[:5])
print(np.round(proba, decimals=3))

[[0.    0.014 0.841 0.    0.136 0.007 0.003]
 [0.    0.003 0.044 0.    0.007 0.946 0.   ]
 [0.    0.    0.034 0.935 0.015 0.016 0.   ]
 [0.011 0.034 0.306 0.007 0.567 0.    0.076]
 [0.    0.    0.904 0.002 0.089 0.002 0.001]]


In [36]:
print(lr.coef_.shape, lr.intercept_.shape)

(7, 5) (7,)


In [37]:
print(lr.coef_, lr.intercept_)

[[-1.49001259 -1.02909653  2.59342992  7.70358183 -1.20070797]
 [ 0.19618191 -2.01072007 -3.77974344  6.50492519 -1.99481478]
 [ 3.562807    6.34355461 -8.48969786 -5.75757213  3.79306162]
 [-0.10458363  3.60320663  3.93067948 -3.61740078 -1.7506979 ]
 [-1.40060998 -6.0750259   5.25969918 -0.87219289  1.86043812]
 [-1.38528461  1.49213609  1.39225441 -5.67734711 -4.40097409]
 [ 0.6215019  -2.32405484 -0.9066217   1.71600589  3.69369499]] [-0.09204689 -0.26289902  3.25100925 -0.14740759  2.65498221 -6.78787045
  1.38423249]


## 소프트맥스

In [38]:
decision = lr.decision_function(test_scaled[:5])
print(np.round(decision, decimals=2))

[[ -6.5    1.03   5.16  -2.73   3.34   0.33  -0.63]
 [-10.86   1.93   4.77  -2.4    2.98   7.84  -4.26]
 [ -4.34  -6.23   3.17   6.49   2.36   2.42  -3.87]
 [ -0.68   0.45   2.65  -1.19   3.26  -5.75   1.26]
 [ -6.4   -1.99   5.82  -0.11   3.5   -0.11  -0.71]]


In [39]:
from scipy.special import softmax ## scipy 라이브러리의 softmax 함수 모듈을 활용하여 소프트 맥스 함수를 직접 구현할 수 있다.

proba = softmax(decision, axis=1)
print(np.round(proba, decimals=3))

[[0.    0.014 0.841 0.    0.136 0.007 0.003]
 [0.    0.003 0.044 0.    0.007 0.946 0.   ]
 [0.    0.    0.034 0.935 0.015 0.016 0.   ]
 [0.011 0.034 0.306 0.007 0.567 0.    0.076]
 [0.    0.    0.904 0.002 0.089 0.002 0.001]]


In [40]:
proba = softmax(decision, axis=0)
print(np.round(proba, decimals=3))

[[0.003 0.247 0.262 0.    0.239 0.001 0.116]
 [0.    0.603 0.177 0.    0.167 0.995 0.003]
 [0.025 0.    0.036 0.998 0.09  0.004 0.005]
 [0.969 0.138 0.021 0.    0.222 0.    0.769]
 [0.003 0.012 0.504 0.001 0.282 0.    0.108]]
